In [2]:
%run packages.py

/Users/mcbookairdebat/Desktop/Code/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2019-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]


In [5]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [6]:
start_date = '2018-01-01'
end_date = '2019-12-31'
period = '1d'
colone = ['Close']


In [7]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [8]:
raw_data = Enginner.RawData()
raw_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'AOS': Price           Close
 Ticker            AOS
 Date                 
 2018-01-02  61.529999
 2018-01-03  61.900002
 2018-01-04  62.189999
 2018-01-05  62.990002
 2018-01-08  63.290001
 ...               ...
 2019-12-23  47.139999
 2019-12-24  47.500000
 2019-12-26  47.740002
 2019-12-27  47.810001
 2019-12-30  47.570000
 
 [502 rows x 1 columns],
 'AMD': Price           Close
 Ticker            AMD
 Date                 
 2018-01-02  10.980000
 2018-01-03  11.550000
 2018-01-04  12.120000
 2018-01-05  11.880000
 2018-01-08  12.280000
 ...               ...
 2019-12-23  45.459999
 2019-12-24  46.540001
 2019-12-26  46.630001
 2019-12-27  46.180000
 2019-12-30  45.520000
 
 [502 rows x 1 columns],
 'ALB': Price            Close
 Ticker             ALB
 Date                  
 2018-01-02  131.529999
 2018-01-03  131.789993
 2018-01-04  130.220001
 2018-01-05  131.720001
 2018-01-08  137.449997
 ...                ...
 2019-12-23   72.430000
 2019-12-24   72.320000
 2019-12-26   72.

In [24]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)

/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/data_import_enginnering.py:24: FutureWarning: Data

In [27]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)

In [28]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()


/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/64972235.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()


In [12]:
Metric = Pair_Selection(dataframe,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list)

In [13]:
distance = Metric.compute_euclidian_distance()


In [14]:

selected_pairs = Metric.paire_selection(distance,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,pairs=pairs)

In [31]:
data1 = dataframe_normalized.copy()
data2 = raw_dataframe.copy()
# The goal would be to stock every small data frame in a dictionnay to help doing the steps for the reste 
dataframes_dictionnary_norm = {}
dataframes_dictionnary_raw ={}
for sector in sector_list: 
    for value in selected_pairs[sector]:
        
        dataframes_dictionnary_norm[value] = data1[list(value)]
        #dataframes_dictionnary_raw[value] = data2[list(value)]

In [34]:
from Spread import Spread


In [35]:
compute_spread = Spread(dataframes_dictionnary_norm)
x = compute_spread.dollar_neutral_spread()

/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]]/values[key[1]])
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/Users/mcbookairdebat/Desktop/Code/project/Pair_trading_strategy/Spread.py:15: SettingWithCo

In [36]:
x

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.600364  0.635466    2.160849
 2019-12-24  0.588530  0.622155    1.850249
 2019-12-26  0.615840  0.645341    1.841183
 2019-12-27  0.609012  0.631601    1.788587
 2019-12-30  0.583068  0.603263    1.470415
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314614         NaN
 2018-01-08  0.174384  0.332054         NaN
 ...              ...       ...         ...
 2019-12-23

In [33]:
for key,values in dataframes_dictionnary_norm.items():
    values['Delta'] = np.abs(values[key[0]] - values[key[1]])
    values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
    values.drop(['Delta'], axis = 1, inplace=True )

dataframes_dictionnary_norm


/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]] - values[key[1]])
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_43808/1693056066.py:4: Se

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.600364  0.635466   -2.613264
 2019-12-24  0.588530  0.622155   -1.971344
 2019-12-26  0.615840  0.645341   -1.857730
 2019-12-27  0.609012  0.631601   -1.897010
 2019-12-30  0.583068  0.603263   -1.593062
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314614         NaN
 2018-01-08  0.174384  0.332054         NaN
 ...              ...       ...         ...
 2019-12-23